<a href="https://colab.research.google.com/github/JimKing100/Jestimate/blob/master/Multiplier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install category_encoders==2.0.0

In [0]:
# Import libraries
import pandas as pd
import numpy as np
import math

import pandas_profiling

from datetime import datetime

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report

import statsmodels.api as sm

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from geopy.distance import vincenty as get_geodesic_distance

In [0]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/SF-SFR-Sales-Final2d.csv')

# Rename subdistr_desc to neighborhood
df = df.rename(columns={'subdist_no': 'nid', 'subdist_desc': 'neighborhood'})

# Fill house square foot zero values with the average house square footage by bedroom for all single family homes in SF
averagesf_data = df.groupby('beds').sf.mean()

# Use average sf by bedroom for each 0 value in each bedroom group up to 9 bedrooms
for i in range(0, 9): 
  df.loc[(df['sf'] == 0) & (df['beds'] == i), 'sf'] = averagesf_data.loc[i]

# Use 10,000sf for anything over 9 bedrooms
df.loc[df['sf'] == 0, 'sf'] = 10000
df = df.astype({'sf': int})

# Check the data
print(df.shape)
df.head(5)

(23711, 39)


,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_suffix,zip,area,district_no,district_desc,nid,neighborhood,on_market_date,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,3/14/13,4,1095000,3/22/13,1260000,8,3.5,4,T,Per Tax Records,2691,NaN,0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN"
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,12/3/15,144,1250000,4/25/16,1075000,9,3.0,4,T,Per Tax Records,2437,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,5/18/17,36,1395000,8/17/17,1525000,9,5.0,5,D,Per Architect,2597,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/27/10,42,725000,8/20/10,715000,5,1.0,2,T,Per Tax Records,1312,NaN,0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK"
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/7/12,24,1595000,7/13/12,1595000,8,4.0,4,T,Per Tax Records,3307,NaN,0,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK"


In [0]:
#pandas_profiling.ProfileReport(df)

In [0]:
# Train, test split on date of 01/01/2018
df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True)
df['year_sold'] = df['sale_date'].dt.year
df['month_sold'] = df['sale_date'].dt.month
df['day_sold'] = df['sale_date'].dt.day

df['on_market_date'] = pd.to_datetime(df['on_market_date'], infer_datetime_format=True)
df['year_on_market'] = df['on_market_date'].dt.year
df['month_on_market'] = df['on_market_date'].dt.month
df['day_on_market'] = df['on_market_date'].dt.day

low_cutoff = 2008
high_cutoff = 2018
train = df[(df['year_sold'] >= low_cutoff) & (df['year_sold'] < high_cutoff)]
test  = df[df['year_sold'] >= high_cutoff]
print(train.shape)
print(test.shape)

(21487, 45)
(2224, 45)


In [0]:
hood_mult = df.groupby(['year_sold']).agg({'sale_price': ['mean']})
hood_mult = hood_mult.set_axis(hood_mult.columns.map('_'.join), axis=1, inplace=False)
hood_mult = hood_mult.reset_index()

base = hood_mult.loc[(hood_mult['year_sold'] == 2018), 'sale_price_mean'].values[0]

hood_mult['multiplier'] = base / hood_mult['sale_price_mean']
hood_mult.head(15)


,year_sold,sale_price_mean,multiplier
0,2009,9.603187e+05,2.042466
1,2010,9.954037e+05,1.970475
2,2011,9.482120e+05,2.068544
3,2012,1.103754e+06,1.777042
4,2013,1.291523e+06,1.518687
5,2014,1.444416e+06,1.357932
6,2015,1.631313e+06,1.202356
7,2016,1.704984e+06,1.150403
8,2017,1.776895e+06,1.103846
9,2018,1.961418e+06,1.000000


In [0]:
tester = hood_mult.loc[(hood_mult['year_sold'] == 2017), 'multiplier'].iloc[0]
tester

1.103846254675958

In [0]:
# hood_mean = mean_hood['sale_price_mean'].loc[(mean_hood['year_sold_'] == 2018) & (mean_hood['nid_'] == 1010)][0]
# hood_mean

# hood_mean = mean_hood.loc[(mean_hood['year_sold_'] == 2018) & (mean_hood['nid_'] == 1010), 'sale_price_mean'].iloc[0]
# hood_mean

In [0]:
# Wrangle the data for train and test
def engineer_features(X):
  
  # Impute mean for null long/lat/elev based on mean of of neighborhood
  def feature_calc(feature, nid, f_dict):
    if math.isnan(feature):
      if (nid) in f_dict:
        new_feature = f_dict[nid]
        return new_feature
    else:
      return feature
    
    return feature
  
  temp = X[~X['longitude'].isna()].groupby(['nid'])['longitude'].mean()
  long_dict = dict(temp)
  X['longitude'] = X.apply(lambda x: feature_calc(x['longitude'], x['nid'], long_dict), axis=1)
  
  temp = X[~X['latitude'].isna()].groupby(['nid'])['latitude'].mean()
  lat_dict = dict(temp)
  X['latitude'] = X.apply(lambda x: feature_calc(x['latitude'], x['nid'], lat_dict), axis=1)
  
  temp = X[~X['elevation'].isna()].groupby(['nid'])['elevation'].mean()
  elev_dict = dict(temp)
  X['elevation'] = X.apply(lambda x: feature_calc(x['elevation'], x['nid'], elev_dict), axis=1)
  
  X['zip'] = X['zip'].astype(int)
  
  # Fill rooms zero values by adding beds and baths
  def room_calc(rooms_val, beds_val, baths_val):
    if rooms_val == 0:
      total = beds_val + baths_val
    else:
      total = rooms_val
      
    return total
  
  X['rooms'] = X.apply(lambda x: room_calc(x['rooms'], x['beds'], x['baths']), axis=1)
  
  # Fill baths zero values by adding beds and baths
  X.loc[(X['baths'] == 0), 'baths'] = 1
  
  # Fill lot_sf zero values by using lot_acres to calc
  def lotsf_calc(lotsf_val, lotacres_val):
    if lotsf_val == 0:
      total = lotacres_val * 43560
    else:
      total = lotsf_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  
  # Fill lot_acres zero values by using lot_sf to calc
  def lotacres_calc(lotacres_val, lotsf_val):
    if lotacres_val == 0:
      total = lotsf_val / 43560
    else:
      total = lotacres_val
      
    return total
  
  X['lot_acres'] = X.apply(lambda x: lotacres_calc(x['lot_acres'], x['lot_sf']), axis=1)
  
  X['ds_count'] = X.apply(lambda x: (x['drive_side'].count(',') + 1), axis=1)
  X['parking_count'] = X.apply(lambda x: (x['parking'].count(',') + 1), axis=1)
  X['view_count'] = X.apply(lambda x: (x['views'].count(',') + 1), axis=1)
  
  X['price_sf'] = X['sale_price'] / X['sf']
  
  mean_hood = X.groupby(['year_sold', 'nid']).agg({'price_sf': ['mean']}).reset_index()
  mean_hood = mean_hood.set_axis(mean_hood.columns.map('_'.join), axis=1, inplace=False)
  mean_hood = mean_hood.reset_index(level=[0])
  
  def hood_mean(yr, hood_id):
    hood_mean = mean_hood.loc[(mean_hood['year_sold_'] == yr) & (mean_hood['nid_'] == hood_id), 'price_sf_mean'].iloc[0]
    adj_hood_mean = (hood_mult.loc[(hood_mult['year_sold'] == yr), 'multiplier'].iloc[0]) * hood_mean
    return adj_hood_mean
  
  X['hood_price_sf'] = X.apply(lambda x: hood_mean(x['year_sold'], x['nid']), axis=1)
  
  # Engineer new feature mean_neighbor_price THIS TAKES 1.5 HOURS TO RUN
  # Output was saved to nhoods files and is loaded later to save time
#   nhoods = X[['sf', 'longitude', 'latitude']]
  
#   def neighbor_mean(sqft, source_latitude, source_longitude):
    
#     source_latlong = source_latitude, source_longitude
#     source_table = X[(X['sf'] >= (sqft * .9)) & (X['sf'] <= (sqft * 1.1))]
#     target_table = pd.DataFrame(source_table, columns = ['latitude', 'longitude', 'year_sold', 'sale_price']) 

#     def get_distance(row):
#         target_latlong = row['latitude'], row['longitude']
#         return get_geodesic_distance(target_latlong, source_latlong).meters

#     target_table['distance'] = target_table.apply(get_distance, axis=1)

#     # Get the nearest 3 locations
#     nearest_target_table = target_table.sort_values(['year_sold', 'distance'], ascending=[False, True])[1:4]

#     return nearest_target_table['sale_price'].mean()/sqft
  
#   # This is commented out for time reasons (see above)
#   nhoods['mean_hood_ppsf'] = X.apply(lambda x: neighbor_mean(x['sf'], x['latitude'], x['longitude']), axis=1)
#   nhoods.to_csv('/content/nhoods.csv')
  
  # Drop unneeded columns
  unneeded_columns = ['sale_date', 'on_market_date', 'city', 'state', 'street_no', 'street_name', 'street_suffix',
                      'day_on_market', 'month_on_market', 'year_on_market', 'month_sold', 'day_sold', 'orig_list_price', 'cdom',
                      'sf_source', 'area', 'sf_source_decs', 'lot_acres', 'views', 'parking', 'zoning', 'price_sf']
  X = X.drop(columns=unneeded_columns)
  
  return X

pd.set_option('mode.chained_assignment', None)
train = engineer_features(train)
test = engineer_features(test)

train.head()

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,lot_desc,drive_side,park_leased,num_parking,shopping,transportation,type,year_sold,ds_count,parking_count,view_count,hood_price_sf
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1260000,8.0,3.5,4,2691,NaN,1969,"RGLR,FNCD","PVDW,PVSW",0,2,4BLK,1BLK,3STR,2013,2,2,4,875.718098
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1075000,9.0,3.0,4,2437,3000.0,1947,RGLR,0,ONST,1,2BLK,1BLK,0,2016,1,4,1,932.452872
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1525000,9.0,5.0,5,2597,3000.0,1947,RGLR,0,ONST,1,2BLK,1BLK,0,2017,1,4,1,1005.849508
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,715000,5.0,1.0,2,1312,NaN,1939,RGLR,"PVDW,PVSW",0,2,3BLK,1BLK,"ATAC,2STR,FIXR",2010,2,1,3,950.268794
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1595000,8.0,4.0,4,3307,NaN,1951,RGLR,PVDW,0,3,4BLK,2BLK,3STR,2012,1,4,4,828.441558


In [0]:
# Load the nhoods data created in the wrangle function - this is the comparable sale price per square foot
# rnhoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/nhoods-train.csv')
# tnhoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/nhoods-test.csv')
# train = pd.merge(train, rnhoods[['old_index', 'mean_hood_ppsf']], left_index=True, right_on='old_index')
# test = pd.merge(test, tnhoods[['old_index', 'mean_hood_ppsf']], left_index=True, right_on='old_index')
# train = train.drop(columns=['old_index'])
# test = test.drop(columns=['old_index'])
# train = train.rename(columns={'mean_hood_ppsf': 'comp_price_sf'})
# test = test.rename(columns={'mean_hood_ppsf': 'comp_price_sf'})

In [0]:
print(train.shape)
train.head()

(21487, 28)


,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,lot_desc,drive_side,park_leased,num_parking,shopping,transportation,type,year_sold,ds_count,parking_count,view_count,hood_price_sf
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1260000,8.0,3.5,4,2691,NaN,1969,"RGLR,FNCD","PVDW,PVSW",0,2,4BLK,1BLK,3STR,2013,2,2,4,875.718098
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1075000,9.0,3.0,4,2437,3000.0,1947,RGLR,0,ONST,1,2BLK,1BLK,0,2016,1,4,1,932.452872
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1525000,9.0,5.0,5,2597,3000.0,1947,RGLR,0,ONST,1,2BLK,1BLK,0,2017,1,4,1,1005.849508
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,715000,5.0,1.0,2,1312,NaN,1939,RGLR,"PVDW,PVSW",0,2,3BLK,1BLK,"ATAC,2STR,FIXR",2010,2,1,3,950.268794
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1595000,8.0,4.0,4,3307,NaN,1951,RGLR,PVDW,0,3,4BLK,2BLK,3STR,2012,1,4,4,828.441558


In [0]:
print(test.shape)
test.head()

(2224, 28)


,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,lot_desc,drive_side,park_leased,num_parking,shopping,transportation,type,year_sold,ds_count,parking_count,view_count,hood_price_sf
5,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121-2410",94121,1,SF District 1,1050,1 - Outer Richmond,3310000,8.0,4.0,4,3735,3959.0,1951,0,0,ONST,3,3BLK,1BLK,2STR,2018,1,4,8,917.926961
7,-122.509090,37.759580,23.45,"1434 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1304000,5.0,2.0,3,1427,2639.0,1900,"RGLR,LEVL",PVDW,ONST,1,1BLK,1BLK,"SMAT,3STR",2018,1,1,5,1021.065316
9,-122.509056,37.758010,23.01,"1516 Great Highway, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1830000,7.0,3.0,4,2120,3072.0,1948,"RGLR,OCNF",0,ONST,1,2BLK,2BLK,FULL,2018,1,1,1,1021.065316
15,-122.509040,37.774418,47.10,"739 48th Ave, San Francisco, CA 94121-3209",94121,1,SF District 1,1050,1 - Outer Richmond,1650000,6.0,2.0,3,1427,3000.0,1957,0,0,ONST,1,0,0,0,2018,1,3,1,917.926961
17,-122.509020,37.758636,22.70,"1492 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1300000,3.0,1.0,2,1087,1245.0,1944,0,"PVDW,PVSW",ONST,1,1BLK,1BLK,0,2018,2,1,1,1021.065316


In [0]:
#pandas_profiling.ProfileReport(train)

In [0]:
# Train
cutoff = 2017
temp=train.copy()
train = temp[temp['year_sold'] < 2017]
val  = temp[temp['year_sold'] >= 2017]
print(train.shape, val.shape, test.shape)

(19226, 28) (2261, 28) (2224, 28)


In [0]:
# Encode and fit a linear regression model
target = 'sale_price'

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
  ce.OrdinalEncoder(),
  SimpleImputer(strategy='mean'), 
  LinearRegression()
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)

# Print regression metrics for validation 
val_mse = mean_squared_error(y_val, y_pred)
val_rmse = np.sqrt(val_mse)
val_mae = mean_absolute_error(y_val, y_pred)
val_r2 = r2_score(y_val, y_pred)
print('Validation Mean Absolute Error:', val_mae)
print('Validation R^2:', val_r2)
print('\n')

ty_pred = pipeline.predict(X_test)

# Print regression metrics for test
test_mse = mean_squared_error(y_test, ty_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, ty_pred)
test_r2 = r2_score(y_test, ty_pred)
print('Test Mean Absolute Error:', test_mae)
print('Test R^2:', test_r2)

Validation Mean Absolute Error: 393211.7059352949
Validation R^2: 0.7109589786796389


Test Mean Absolute Error: 429785.1117131287
Test R^2: 0.6214007271801465


In [0]:
X_train.head()

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,rooms,baths,beds,sf,lot_sf,year_built,lot_desc,drive_side,park_leased,num_parking,shopping,transportation,type,year_sold,ds_count,parking_count,view_count,hood_price_sf
0,-122.509650,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,8.0,3.5,4,2691,NaN,1969,"RGLR,FNCD","PVDW,PVSW",0,2,4BLK,1BLK,3STR,2013,2,2,4,875.718098
1,-122.509290,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,9.0,3.0,4,2437,3000.0,1947,RGLR,0,ONST,1,2BLK,1BLK,0,2016,1,4,1,932.452872
3,-122.509240,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,5.0,1.0,2,1312,NaN,1939,RGLR,"PVDW,PVSW",0,2,3BLK,1BLK,"ATAC,2STR,FIXR",2010,2,1,3,950.268794
4,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,8.0,4.0,4,3307,NaN,1951,RGLR,PVDW,0,3,4BLK,2BLK,3STR,2012,1,4,4,828.441558
6,-122.509186,37.761005,23.69,"1362 La Playa St, San Francisco, CA 94122-1019",94122,2,SF District 2,2030,2 - Outer Sunset,4.0,1.0,3,1168,NaN,1947,"RGLR,LEVL",PVDW,0,2,0,1BLK,ATAC,2009,1,3,1,1058.267767


In [0]:
# Add the prediction and difference from the actual sales price
final = test.copy()
final['prediction'] = ty_pred
final['difference'] = final['prediction'] - final['sale_price']
final['prediction'] = final['prediction'].astype(int)
final['difference'] = final['difference'].astype(int)
print(final.shape)
final.head()

(2224, 30)


,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,lot_desc,drive_side,park_leased,num_parking,shopping,transportation,type,year_sold,ds_count,parking_count,view_count,hood_price_sf,prediction,difference
5,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121-2410",94121,1,SF District 1,1050,1 - Outer Richmond,3310000,8.0,4.0,4,3735,3959.0,1951,0,0,ONST,3,3BLK,1BLK,2STR,2018,1,4,8,917.926961,3488940,178940
7,-122.509090,37.759580,23.45,"1434 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1304000,5.0,2.0,3,1427,2639.0,1900,"RGLR,LEVL",PVDW,ONST,1,1BLK,1BLK,"SMAT,3STR",2018,1,1,5,1021.065316,1815647,511647
9,-122.509056,37.758010,23.01,"1516 Great Highway, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1830000,7.0,3.0,4,2120,3072.0,1948,"RGLR,OCNF",0,ONST,1,2BLK,2BLK,FULL,2018,1,1,1,1021.065316,2108920,278920
15,-122.509040,37.774418,47.10,"739 48th Ave, San Francisco, CA 94121-3209",94121,1,SF District 1,1050,1 - Outer Richmond,1650000,6.0,2.0,3,1427,3000.0,1957,0,0,ONST,1,0,0,0,2018,1,3,1,917.926961,1537258,-112741
17,-122.509020,37.758636,22.70,"1492 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1300000,3.0,1.0,2,1087,1245.0,1944,0,"PVDW,PVSW",ONST,1,1BLK,1BLK,0,2018,2,1,1,1021.065316,1084350,-215649


In [0]:
# Calculate the metrics used by Zillow
final['pred_percent'] = final['difference']/final['prediction']
pred_median_error = final['pred_percent'].median()
pred_five_percent = (final['pred_percent'][(final['pred_percent'] >= -.05) &
                                           (final['pred_percent'] <= .05)].count())/final.shape[0]

pred_ten_percent = (final['pred_percent'][(final['pred_percent'] >= -.10) &
                                          (final['pred_percent'] <= .10)].count())/final.shape[0]

pred_twenty_percent = (final['pred_percent'][(final['pred_percent'] >= -.20) &
                                             (final['pred_percent'] <= .20)].count())/final.shape[0]

print('Median Error - %.4f%%' % (pred_median_error * 100))
print('Prediction Within 5 percent - %.4f%%' % (pred_five_percent * 100))
print('Prediction Within 10 percent - %.4f%%' % (pred_ten_percent * 100))
print('Prediction Within 20 percent - %.4f%%' % (pred_twenty_percent * 100))

Median Error - 8.7170%
Prediction Within 5 percent - 16.3219%
Prediction Within 10 percent - 33.5432%
Prediction Within 20 percent - 61.1960%


In [0]:
# data = final[['full_address', 'neighborhood', 'nid', 'sale_price', 'prediction', 'difference']]
# data = data.drop_duplicates()

# data.head()

In [0]:
# data.shape

In [0]:
# data.to_csv('/content/display_data.csv')